In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
elements = spark.read.csv(
    "../../data/elements/Periodic_Table_Of_Elements.csv",
    header=True,
    inferSchema=True
)

In [5]:
(elements
 .where(F.col("phase") == "liq")
 .groupby("period")
 .count()
 .show())

+------+-----+
|period|count|
+------+-----+
|     6|    1|
|     4|    1|
+------+-----+



SQL equivalent for this operation is:
```
SELECT
 period,
 count(*)
FROM elements
WHERE phase = 'liq'
GROUP BY period;
```

PySpark maintains boundaries between its own name spacing and Spark SQL’s name spacing so we have to explicitly promote them.

In [6]:
try:
    spark.sql(
        "select period, count(*) from elements "
        "where phase='liq' group by period"
    ).show(5)
except AnalysisException as e:
    print(e)

[TABLE_OR_VIEW_NOT_FOUND] The table or view `elements` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 29;
'Aggregate ['period], ['period, unresolvedalias(count(1), None)]
+- 'Filter ('phase = liq)
   +- 'UnresolvedRelation [elements], [], false



To allow a data frame to be queried via SQL, we need to _register_ it using `createOrReplaceTempView`.

In [7]:
elements.createOrReplaceTempView("elements")

In [8]:
spark.sql(
    "select period, count(*) from elements "
    "where phase='liq' group by period"
).show(5)

+------+--------+
|period|count(1)|
+------+--------+
|     6|       1|
|     4|       1|
+------+--------+



We can use the catalog to list the tables/views we have registered and drop them if we are done.

In [9]:
spark.catalog.listTables()

[Table(name='elements', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [10]:
spark.catalog.dropTempView("elements")
spark.catalog.listTables()

[]

Backblaze is a company that provides cloud storage and backup. Since 2013, they have provided data on the drives in
their data center, and over time have moved to a focus on failures and diagnosis.

All data are available from https://www.backblaze.com/cloud-storage/resources/hard-drive-test-data.

To avoid blowing our memory, we'll work with just Q3 2019.

Since this data is ~3GB when unpacked, it is not checked in to this repo.

In [44]:
DATA_DIRECTORY = "../../data/backblaze/data_Q3_2019"

q3 = spark.read.csv(
    DATA_DIRECTORY,
    header=True,
    inferSchema=True
)

backblaze_2019 = q3

# Set the layour for each column according to the schema

backblaze_2019 = backblaze_2019.select(
    [
        F.col(x).cast(T.LongType()) if x.startswith("smart") else F.col(x)
        for x in backblaze_2019.columns
    ]
)

backblaze_2019.createOrReplaceTempView("backblaze_stats_2019")

In [13]:
# Show hard drive serial numbers that have failed as some point:
spark.sql(
    "select serial_number from backblaze_stats_2019 where failure = 1"
).show(5)

+-------------+
|serial_number|
+-------------+
|     ZA10MCJ5|
|     ZCH07T9K|
|     ZCH0CA7Z|
|     Z302F381|
|     ZCH0B3Z2|
+-------------+
only showing top 5 rows



In [16]:
# Get the capacity in gigabytes of the hardrives in the data
spark.sql(
    """SELECT
            model,
            min(capacity_bytes / pow(1024, 3)) AS min_GB,
            max(capacity_bytes / pow(1024, 3)) AS max_GB
        FROM backblaze_stats_2019
        GROUP BY 1
        ORDER BY 3 DESC
        """
).show(5)

+--------------------+--------------------+-------+
|               model|              min_GB| max_GB|
+--------------------+--------------------+-------+
| TOSHIBA MG07ACA14TA|             13039.0|13039.0|
|       ST12000NM0007|-9.31322574615478...|11176.0|
|HGST HUH721212ALE600|             11176.0|11176.0|
|       ST12000NM0117|             11176.0|11176.0|
|HGST HUH721212ALN604|-9.31322574615478...|11176.0|
+--------------------+--------------------+-------+
only showing top 5 rows



In [17]:
# Get the capacity in gigabytes of the hardrives in the data
backblaze_2019.groupby(F.col("model")).agg(
    F.min(F.col("capacity_bytes") / F.pow(F.lit(1024), 3)).alias("min_GB"),
    F.max(F.col("capacity_bytes") / F.pow(F.lit(1024), 3)).alias("max_GB")
).orderBy(F.col("max_GB"), ascending=False).show(5)

+--------------------+--------------------+-------+
|               model|              min_GB| max_GB|
+--------------------+--------------------+-------+
| TOSHIBA MG07ACA14TA|             13039.0|13039.0|
|       ST12000NM0007|-9.31322574615478...|11176.0|
|HGST HUH721212ALE600|             11176.0|11176.0|
|       ST12000NM0117|             11176.0|11176.0|
|HGST HUH721212ALN604|-9.31322574615478...|11176.0|
+--------------------+--------------------+-------+
only showing top 5 rows



Looking at the results from our query, there are some drives that report more than one capacity. Furthermore, we have some drives that report negative capacity, which is really odd. Let’s focus on seeing how prevalent this is.

We assume that the maximum reported capacity for each model is the correct one.

In [18]:
# How many models have more than one capacity?
spark.sql(
    """
    SELECT
        model,
        min(capacity_bytes / pow(1024, 3)) AS min_GB,
        max(capacity_bytes / pow(1024, 3)) AS max_GB
    FROM backblaze_stats_2019
    GROUP BY 1
    HAVING min_GB != max_GB
    ORDER BY 3 DESC
"""
).show(5)

+--------------------+--------------------+-----------------+
|               model|              min_GB|           max_GB|
+--------------------+--------------------+-----------------+
|       ST12000NM0007|-9.31322574615478...|          11176.0|
|HGST HUH721212ALN604|-9.31322574615478...|          11176.0|
|HGST HUH721010ALE600|-9.31322574615478...|           9314.0|
|       ST10000NM0086|-9.31322574615478...|           9314.0|
|        ST8000NM0055|-9.31322574615478...|7452.036460876465|
+--------------------+--------------------+-----------------+
only showing top 5 rows



In [19]:
backblaze_2019.createOrReplaceTempView("drive_stats")

In [25]:
spark.sql(
    """
    CREATE OR REPLACE TEMP VIEW drive_days AS
        SELECT model, count(*) AS drive_days
        FROM drive_stats
        GROUP BY model
    """
)

spark.sql(
    """
    CREATE OR REPLACE TEMP VIEW failures AS
        SELECT model, count(*) AS failures
        FROM drive_stats
        WHERE failure = 1
        GROUP BY model
    """
)

DataFrame[]

In [26]:
drive_days = (
    backblaze_2019.groupby(F.col("model"))
    .agg(F.count(F.col("*")).alias("drive_days"))
)

failures = (
    backblaze_2019.where(F.col("failure") == 1)
    .groupby(F.col("model"))
    .agg(F.count(F.col("*")).alias("failures"))
)

In [28]:
spark.sql(
    """
    SELECT drive_days.model, drive_days, failures
    FROM drive_days
    LEFT JOIN failures
    ON drive_days.model = failures.model
    """
).show(5)

+-------------+----------+--------+
|        model|drive_days|failures|
+-------------+----------+--------+
|  ST9250315AS|        89|    null|
|  ST4000DM000|   1796728|      72|
|ST12000NM0007|   3212635|     361|
|  ST8000DM005|      2280|       1|
|   ST320LT007|        89|    null|
+-------------+----------+--------+
only showing top 5 rows



In [29]:
# SUBQUERIES
# Find the models with the highest failure rates in Q3 2019
# by querying drive_stats directly, using subqueries
spark.sql(
    """
    SELECT failures.model, failures / drive_days AS failure_rate
    FROM (
        SELECT model, count(*) AS drive_days
        FROM drive_stats
        GROUP BY model
    ) AS drive_days
    INNER JOIN (
        SELECT model, count(*) AS failures
        FROM drive_stats
        WHERE failure = 1
        GROUP BY model
    ) AS failures
    ON drive_days.model = failures.model
    ORDER BY 2 desc
    """
).show(5)

+------------------+--------------------+
|             model|        failure_rate|
+------------------+--------------------+
|     ST12000NM0117|0.019305019305019305|
|TOSHIBA MQ01ABF050|5.579360828423496E-4|
|       ST8000DM005|4.385964912280702E-4|
|        ST500LM030| 4.19639110365086E-4|
|     ST500LM012 HN|1.511585221015353...|
+------------------+--------------------+
only showing top 5 rows



In [35]:
# CTEs
# Find the models with the highest failure rates in Q3 2019
# by querying drive_stats directly, using subqueries CTEs
spark.sql(
    """
    WITH drive_days AS (
        SELECT model, count(*) AS drive_days
        FROM drive_stats
        GROUP BY model
    ),
    failures AS (
        SELECT model, count(*) AS failures
        FROM drive_stats
        WHERE failure = 1
        GROUP BY model
    )
    SELECT failures.model, failures / drive_days AS failure_rate
    FROM drive_days 
    INNER JOIN failures
    ON drive_days.model = failures.model
    ORDER BY 2 desc
    """
).show(5)

+------------------+--------------------+
|             model|        failure_rate|
+------------------+--------------------+
|     ST12000NM0117|0.019305019305019305|
|TOSHIBA MQ01ABF050|5.579360828423496E-4|
|       ST8000DM005|4.385964912280702E-4|
|        ST500LM030| 4.19639110365086E-4|
|     ST500LM012 HN|1.511585221015353...|
+------------------+--------------------+
only showing top 5 rows



In [36]:
# Python function
# Find the models with the highest failure rates in Q3 2019
# by querying drive_stats directly

def failure_rate(drive_stats):
    drive_days = drive_stats.groupby(F.col("model")).agg(
        F.count(F.col("*")).alias("drive_days")
    )

    failures = (
        drive_stats.where(F.col("failure") == 1)
        .groupby(F.col("model"))
        .agg(F.count(F.col("*")).alias("failures"))
    )

    answer = (
        drive_days.join(failures, on="model", how="inner")
        .withColumn("failure_rate", F.col("failures") / F.col("drive_days"))
        .orderBy(F.col("failure_rate").desc())
    )

    return answer

In [37]:
failure_rate(backblaze_2019).show(5)

+------------------+----------+--------+--------------------+
|             model|drive_days|failures|        failure_rate|
+------------------+----------+--------+--------------------+
|     ST12000NM0117|       259|       5|0.019305019305019305|
|TOSHIBA MQ01ABF050|     44808|      25|5.579360828423496E-4|
|       ST8000DM005|      2280|       1|4.385964912280702E-4|
|        ST500LM030|     21447|       9| 4.19639110365086E-4|
|     ST500LM012 HN|     46309|       7|1.511585221015353...|
+------------------+----------+--------+--------------------+
only showing top 5 rows



### Exercise 7.1

Taking the elements data frame, which PySpark code is equivalent to the following SQL statement?

```
select count(*) from elements where Radioactive is not null;
```

In [38]:
elements.where(F.col("Radioactive").isNotNull()).count()

37

Using SQL-style expressions in PySpark

We can take an alternative approach where we pre-process our data so that it more easily answers our question:

In [39]:
full_data = backblaze_2019.selectExpr(
    "model", "capacity_bytes / pow(1024, 3) AS capacity_GB", "date", "failure"
)

drive_days = full_data.groupby("model", "capacity_GB").agg(
    F.count("*").alias("drive_days")
)

failures = (
    full_data.where("failure = 1")
    .groupby("model", "capacity_GB")
    .agg(F.count("*").alias("failures"))
)

summarised_data = (
    drive_days.join(failures, on=["model", "capacity_GB"], how="left")
    .fillna(0.0, ["failures"])
    .selectExpr("model", "capacity_GB", "failures / drive_days AS failure_rate")
)

In [42]:
def most_reliable_drive_for_capacity(data, 
                                     capacity_GB=2048, 
                                     precision=0.25,
                                     top_n=3):
    capacity_min = capacity_GB / (1 + precision)
    capacity_max = capacity_GB * (1 + precision)

    answer = (
        data.where(f"capacity_GB between {capacity_min} and {capacity_max}")
        .orderBy("failure_rate", "capacity_GB", ascending=[True, True])
        .limit(top_n)
    )

    return answer

In [43]:
most_reliable_drive_for_capacity(summarised_data, capacity_GB=11176.0).show()

+--------------------+-----------+--------------------+
|               model|capacity_GB|        failure_rate|
+--------------------+-----------+--------------------+
|HGST HUH721010ALE600|     9314.0|                 0.0|
|HGST HUH721212ALN604|    11176.0|1.585588480593982...|
|HGST HUH721212ALE600|    11176.0|1.636661211129296...|
+--------------------+-----------+--------------------+

